In [1]:
import pandas as pd
import numpy as np

def load_mf(fp, index='#SampleID'):
    _df = pd.read_csv(fp, sep='\t', keep_default_na=False, na_values=[], dtype=str)
    _df.set_index(index, inplace=True)
    return _df

Download the mapping files for the ICU, ITS and FMT from Qiita. Then combine them with the AG mapping file that you got from Justine.

In [2]:
mf = load_mf('ag_full_no_letters.fmt-icu-its.tsv')

Create two columns for the ITS animation.

In [3]:
def rename(row):
    if row.qiita_study_id == '101' and row.name != '101.Mother':
        return 'Child'
    else:
        return 'no_data'
mf['animations_trajectory_its'] = mf.apply(rename, axis=1, reduce=False)

def renumber(row):
    if row.qiita_study_id == '101' and row.name != '101.Mother':
        return str(int(row.age))
    else:
        return '0'
mf['animations_gradient_its'] = mf.apply(renumber, axis=1, reduce=False)

Now add a run number column.

In [4]:
# Run number additions
date = pd.to_datetime(mf['run_date'])
date.loc[date == pd.to_datetime('2013-02-25')] = pd.to_datetime('2013-01-01')

date_count = pd.DataFrame(date.value_counts().sort_index())
date_count['order'] = np.arange(0, len(date_count))
order_map = date_count[['order']].to_dict()
run_order = date.apply(lambda x: order_map['order'][x])

mf['run_number'] = run_order.copy()

The values for samples from the ITS, FMT and ICU should not be included in this column.

In [5]:
def renumber_runs(row):
    # American Gut
    if row.qiita_study_id == '10317':
        return row.run_number
    else:
        # FMT and ITS
        if row.qiita_study_id in {'101', '1924'}:
            return 'FMT and ITS'
        # ICU
        else:
            return 'ICU'

mf['run_number'] = mf.apply(renumber_runs, axis=1, reduce=False)

To make the animation a bit smoother, we will make two columns that capture higher level groups in the `COUNTRY` and `BODY_PRODUCT` categories.

In [6]:
rename_body_product = {'UBERON:cerumen': 'Skin',
 'UBERON:feces': 'Fecal',
 'UBERON:mucus': 'Oral',
 'UBERON:saliva': 'Oral',
 'UBERON:sebum': 'Skin',
 'UBERON:tears': 'Oral',
 'not applicable': 'Fecal'}

mf['body_product_animation'] = mf.body_product.replace(rename_body_product)

def rename_country(row):
    if row.country in {'USA', 'United Kingdom', 'Australia'}:
        return row.country
    else:
        return 'Other'

mf['country_animation'] = mf.apply(rename_country, axis=1, reduce=False)

In [8]:
mf.to_csv('ag_full_no_letters.its-and-fmt.with-animations.tsv', sep='\t')